In [182]:
import requests
import time
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np

### 웹 페이지 유형
- 정적(static) 웹페이지: 저장되어있는 html, css를 그대로 사용하여 렌더링이 항상 일정한 페이지
- 동적(dynamic) 웹페이지: 클라이언트의 요청, 외부 변수에 따라 다르게 렌더링 되는 페이지


### 크롤링 유형

||정적 크롤링|동적 크롤링|
|---|---|---|
|정보 수집 라이브러리|requests, urllib|selenium|
|html 파싱 라이브러리|bs4|bs4, selenium|
|페이지 조작|X|O|
|속도|빠름|느림||

## Request 모듈
- requests는 python용 HTTP 요청 모듈
- 웹브라우저에서 특정 홈페이지에 접속하는 역할을 하여 정보를 받아옴

In [1]:
!pip install requests

In [2]:
import requests

In [3]:
res = requests.get('http://naver.com') #해당 사이트의 HTML 문서를 받아옴
# res.raise_for_status()
res.status_code # 응답코드 200이면 정상

200

In [4]:
res = requests.get('https://kr.investing.com/equities/apple-computer-inc-historical-data')
res.status_code

403

### User-Agent
- 웹사이트에서는 접속하는 사용자의 정보를 알 수 있음 (header 정보)
- 스마트폰에서 접속할 때와 데스크탑에서 접속을 할 때 각자 다른 웹사이트가 나타남
- 사람이 접속하지 않고 웹크롤러가 접속을 할 경우 서버에 부하가 걸릴 수 있기 때문에 정보를 확인해서 차단 가능
- 이러한 경우 User-Agent를 이용해서 해결 가능


- what is my user agent?

In [5]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

In [6]:
url = 'https://kr.investing.com/equities/apple-computer-inc-historical-data'

In [7]:
res = requests.get(url, headers = header)

In [8]:
# 정상적으로 접속
res.status_code

200

In [9]:
res.text

' <!DOCTYPE HTML>\n<html dir="ltr" xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" xmlns:schema="http://schema.org/" class="kr" lang="ko" geo="KR">\n<head>\n<link rel="dns-prefetch" href="https://i-invdn-com.investing.com" />\n<link rel="dns-prefetch" href="https://a-invdn-com.investing.com" />\n<script>\n        window.dataLayerGTMParams = window.dataLayerGTMParams || [];\n        dataLayerGTMParams.push({"client_name":"legacy-desktop"});\n    </script>\n<script>\n                dataLayerGTMParams.push({"instrument_id":"6408"});\n            </script>\n\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n                new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\n            j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n            \'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n        })(window,document,\'script\',\'dataLayerGTMParams\',\'GTM-PGT3R3D\');</scri

## 1. 정적 크롤링: BeautifulSoup 기초
- soup.tag.get_text(): tag 안에 있는 정보를 받아옴
- soup.tag.text: tag 안에 있는 정보를 받아옴
- soup.tag.atttrs: tag의 속성 정보를 받아옴
- soup.find('tag', attrs={'class':'class value'): 첫번째로 나오는 'class'값이 'class value'인 'tag'를 찾아서 가져옴
- soup.find_all('tag',attrs={'class':'class value'): 'class'값이 'class value'인 'tag'들을 모두 찾아 리스트 안에 받아옴
- soup.select('경로'): 경로에 있는 tag들을 모두 리스트 안에 받아옴
- soup.select_one('경로'):'경로'에 있는 tag를 찾아 가져옴

In [10]:
!pip install beautifulSoup4
!pip install lxml

In [11]:
import requests
from bs4 import BeautifulSoup

### EXAMPLE 1) 네이버 영화 정보
- 영화의 정보들 가져오기
1. 정보를 가져올 url 찾기
2. url에 있는 HTML 문서를 받아옴
3. HTML 문서를 BeautifulSoup 객체로 변환
4. BeautifulSoup를 통해 원하는 정보 찾아냄

In [12]:
url = 'https://movie.naver.com/movie/sdb/rank/rmovie.naver'

In [13]:
res = requests.get(url)

In [14]:
soup = BeautifulSoup(res.text, 'lxml')

In [15]:
soup

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="me2:image"/>
<meta content="네이버영화 " property="me2:post_tag"/>
<meta content="네이버영화" property="me2:category1"/>
<meta content="" property="me2:category2"/>
<meta content="랭킹 : 네이버 영화" property="og:title"/>
<meta content="영화, 영화인, 예매, 박스오피스 랭킹 정보 제공" property="og:description"/>
<meta content="article" property="og:type"/>
<meta content="https://movie.naver.com/movie/sdb/rank/rmovie.naver" property="og:url"/>
<meta content="http://static.naver.net/m/movie/icons/OG_270_270.png" property="og:image"/><!-- http://static.naver.net/m/movie/im/navermovie.jpg -->
<meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="og:article:thumbnailUrl"/>
<meta content="네이버 영화" property="og:article:author"/>
<meta content="https://m

In [16]:
soup.title

<title>랭킹 : 네이버 영화</title>

In [17]:
soup.title.text

'랭킹 : 네이버 영화'

In [18]:
soup.a.attrs

{'name': 'gnb_top'}

In [19]:
soup.a['name']

'gnb_top'

#### 1) 영화의 id number와 제목 가져오기
- 개발자도구(F12)에서 영화 정보가 있는 태그 탐색
- 태그와 속성값을 찾음

In [22]:
movies = soup.find('table', attrs={'class':'list_ranking'})

In [23]:
movies

<table cellspacing="0" class="list_ranking">
<caption class="blind">랭킹 테이블</caption>
<col width="6%"/><col width="*"/><col width="2%"/><col width="4%"/>
<thead>
<tr>
<th scope="col">순위</th>
<th scope="col">영화명</th>
<th colspan="2" scope="col">변동폭</th>
</tr>
</thead>
<tbody>
<tr><td class="blank01" colspan="8"></td></tr>
<!-- 예제
				<tr>
					<td class="ac"><img src="https://ssl.pstatic.net/imgmovie/2007/img/common/bullet_r_g50.gif" alt="50" width="14" height="13"></td>
					<td class="title"><a href="#">트랜스포머</a></td>
					<td class="ac"><img src="https://ssl.pstatic.net/imgmovie/2007/img/common/icon_down_1.gif" alt="down" width="7" height="10"></td>
					<td class="range ac">7</td>
				</tr>
				-->
<tr>
<td class="ac"><img alt="01" height="13" src="https://ssl.pstatic.net/imgmovie/2007/img/common/bullet_r_r01.gif" width="14"/></td>
<td class="title">
<div class="tit3">
<a href="/movie/bi/mi/basic.naver?code=164122" title="신비한 동물들과 덤블도어의 비밀">신비한 동물들과 덤블도어의 비밀</a>
</div>
</td>
<!-- 평점순

- table 태그와 link_ranking 이라는 속성을 가진 soup object를 movies 로 받아옴
- 가져온 movies도 soup object 이기 때문에 soup의 method들 사용 가능

In [24]:
#'tr'태그를 가진 모든 태그를 리스트의 형태로 가져오기
movie_list = movies.find_all('tr')

In [25]:
movie_list

[<tr>
 <th scope="col">순위</th>
 <th scope="col">영화명</th>
 <th colspan="2" scope="col">변동폭</th>
 </tr>,
 <tr><td class="blank01" colspan="8"></td></tr>,
 <tr>
 <td class="ac"><img alt="01" height="13" src="https://ssl.pstatic.net/imgmovie/2007/img/common/bullet_r_r01.gif" width="14"/></td>
 <td class="title">
 <div class="tit3">
 <a href="/movie/bi/mi/basic.naver?code=164122" title="신비한 동물들과 덤블도어의 비밀">신비한 동물들과 덤블도어의 비밀</a>
 </div>
 </td>
 <!-- 평점순일 때 평점 추가하기  -->
 <!----------------------------------------->
 <td class="ac"><img alt="na" class="arrow" height="10" src="https://ssl.pstatic.net/imgmovie/2007/img/common/icon_na_1.gif" width="7"/></td>
 <td class="range ac">0</td>
 </tr>,
 <tr>
 <td class="ac"><img alt="02" height="13" src="https://ssl.pstatic.net/imgmovie/2007/img/common/bullet_r_r02.gif" width="14"/></td>
 <td class="title">
 <div class="tit3">
 <a href="/movie/bi/mi/basic.naver?code=190695" title="모비우스">모비우스</a>
 </div>
 </td>
 <!-- 평점순일 때 평점 추가하기  -->
 <!----------------

In [26]:
len(movie_list)

57

In [28]:
for movie in movie_list:
    if movie.find('a'):
        print(movie.find('a')['href'])
        print(movie.find('a').text)
        print('-'*50)

/movie/bi/mi/basic.naver?code=164122
신비한 동물들과 덤블도어의 비밀
--------------------------------------------------
/movie/bi/mi/basic.naver?code=190695
모비우스
--------------------------------------------------
/movie/bi/mi/basic.naver?code=190991
이상한 나라의 수학자
--------------------------------------------------
/movie/bi/mi/basic.naver?code=212095
수퍼 소닉2
--------------------------------------------------
/movie/bi/mi/basic.naver?code=214553
앰뷸런스
--------------------------------------------------
/movie/bi/mi/basic.naver?code=189137
스텔라
--------------------------------------------------
/movie/bi/mi/basic.naver?code=154282
더 배트맨
--------------------------------------------------
/movie/bi/mi/basic.naver?code=183862
뜨거운 피
--------------------------------------------------
/movie/bi/mi/basic.naver?code=191020
야차
--------------------------------------------------
/movie/bi/mi/basic.naver?code=182016
닥터 스트레인지: 대혼돈의 멀티버스
--------------------------------------------------
/movie/bi/mi/basic.naver?code=1903

- 영화의 타이틀이 'tr'이라는 태그에 있으므로 find_all 메소드를 통해 모두 가져옴
- find_all은 조건에 만족하는 모든 soup object들을 리스트로 순서대로 받아옴

- 가져온 soup object들의 리스트에서 인덱스 넘버와 제목 추출

In [31]:
movie_id_name = []
for movie in movie_list:
    if movie.find('a'):
        movie_id = movie.find('a')['href'][30:]
        movie_title = movie.find('a').text
        movie_id_name.append((movie_id, movie_title))   ####왜 괄호 두개??

In [32]:
movie_id_name

[('164122', '신비한 동물들과 덤블도어의 비밀'),
 ('190695', '모비우스'),
 ('190991', '이상한 나라의 수학자'),
 ('212095', '수퍼 소닉2'),
 ('214553', '앰뷸런스'),
 ('189137', '스텔라'),
 ('154282', '더 배트맨'),
 ('183862', '뜨거운 피'),
 ('191020', '야차'),
 ('182016', '닥터 스트레인지: 대혼돈의 멀티버스'),
 ('190374', '앵커'),
 ('192608', '범죄도시2'),
 ('201701', '불도저에 탄 소녀'),
 ('201938', '그리스도 디 오리진'),
 ('196854', '브로커'),
 ('209859', '배니싱: 미제사건'),
 ('196362', '공기살인'),
 ('212098', '로스트 시티'),
 ('213733', '극장판 주술회전 0'),
 ('201766', '말임씨를 부탁해'),
 ('159812', '니 부모 얼굴이 보고 싶다'),
 ('213903', '문폴'),
 ('204138', '복지식당'),
 ('193961', '루이스 웨인: 사랑을 그린 고양이 화가'),
 ('215637', '더 컨트랙터'),
 ('211165', '파일럿: 배틀 포 서바이벌'),
 ('154255', '신비한 동물들과 그린델왈드의 범죄'),
 ('215791', '서울괴담'),
 ('198624', '태어나길 잘했어'),
 ('207444', '고스팅 글로리아'),
 ('214569', 'B컷'),
 ('115642', '신비한 동물사전'),
 ('196051', '극장판 귀멸의 칼날: 무한열차편'),
 ('213580', '3.5교시'),
 ('213570', '웨이트: 감각에 눈뜰 때'),
 ('204650', '봄날'),
 ('154485', '4월은 너의 거짓말'),
 ('194196', '한산: 용의 출현'),
 ('208077', '스파이더맨: 노 웨이 홈'),
 ('199164', '행복을 

2) 영화의 다른 정보들 가져와보기

In [33]:
url = 'https://movie.naver.com/movie/bi/mi/basic.naver?code=164122'

In [35]:
res = requests.get(url)

In [36]:
soup = BeautifulSoup(res.text,'lxml')

- soup.select_one('경로') : soup obejct로 받아옴
- soup.select('경로') : 리스트로 받아옴
- 기본 정보를 가져오기 위해서 개발자도구에서 정보의 주소를 찾음 (copy selector)

- 제목

In [ ]:
'#content > div.article > div.mv_info_area > div.mv_info > h3 > a'

In [38]:
soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')

<a href="./basic.naver?code=164122">신비한 동물들과 덤블도어의 비밀</a>

In [39]:
soup.find('h3', {'class': 'h_movie'}).a.text

'신비한 동물들과 덤블도어의 비밀'

- 장르, 국가 정보 가져오기

In [40]:
basic_info = soup.find('dl', {'class':'info_spec'}).find_all('span')

In [42]:
basic_info[0].text

'\n판타지, \r\n\t\t\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t\t\r\n\t\t\t\t\t모험, \r\n\t\t\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t\t\r\n\t\t\t\t\t가족\n'

In [44]:
genre = basic_info[0].text.split()
genre

['판타지,', '모험,', '가족']

In [45]:
basic_info[1]

<span>
<a href="/movie/sdb/browsing/bmovie.naver?nation=US">미국</a><!-- N=a:ifo.nation -->
</span>

In [46]:
basic_info[1].a.text.split()

['미국']

- 상영시간

In [47]:
basic_info[2]

<span>142분 </span>

In [48]:
basic_info[2].text

'142분 '

- 평점 가져오기

In [49]:
star_score = soup.find('div', {'class':'star_score'})
star_score

<div class="star_score">
<span class="st_off"><span class="st_on" style="width:68.7%">관람객 평점 6.87점</span></span><em class="num6">6</em><em class="dot">.</em><em class="num8">8</em><em class="num7">7</em>
</div>

In [51]:
score_audience = float(soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-child(5) > div:nth-child(2) > div.score_area > div.netizen_score > div > div > em').text)
score_audience

6.46

In [52]:
score_critic = float(soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-child(5) > div:nth-child(2) > div.score_area > div.special_score > div > div > em').text)
score_critic

5.67

### 3) 전체과정

In [56]:
import requests
from bs4 import BeautifulSoup

url = 'https://movie.naver.com/movie/sdb/rank/rmovie.naver'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
movie_list = soup.find('table', {'class': 'list_ranking'}).find_all('tr')

movie_final = [] # 이곳에 영화정보를 받아오게됨

# 영화랭킹 페이지에서 영화의 id(code)와 제목을 가져옴
for movie in movie_list:
    if movie.find('a'):
        movie_id = movie.find('a')['href'][30:]
        movie_title = movie.find('a').text
        
        # 각 영화들의 정보 추출
        url = 'https://movie.naver.com/movie/bi/mi/basic.naver?code={}'.format(movie_id)
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'lxml')
        
        title = soup.find('h3', {'class':'h_movie'}).a.text
        
        # 장르, 국가정보, 상영시간
        basic_info = soup.find('dl', {'class':'info_spec'}).find_all('span')
        
        if len(basic_info) <= 2:
            genre = basic_info[0].text.split() #장르
            nation = basic_info[1].a.text.split() # 국가
            run_time = 'NaN' #상영시간
        else:
            genre = basic_info[0].text.split() #장르
            nation = basic_info[1].a.text.split() # 국가
            run_time = basic_info[2].text
            
        # 평점이 없는 경우가 있음
        if soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-child(5) > div:nth-child(2) > div.score_area > div.netizen_score > div > div > em'):
            score_audience = float(soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-child(5) > div:nth-child(2) > div.score_area > div.netizen_score > div > div > em').text)
        else:
            score_audience = 'Nan'
            
        if soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-child(5) > div:nth-child(2) > div.score_area > div.special_score > div > div > em'):
            score_critic = score_critic = float(soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-child(5) > div:nth-child(2) > div.score_area > div.special_score > div > div > em').text)
        else:
            score_critic = 'NaN'
            
        info = [movie_id, movie_title, genre, nation, score_audience, score_critic, run_time]
        
        movie_final.append(info)
        
        print('{} movie finished'.format(movie_id))
        

print('finished')



164122 movie finished
190695 movie finished
190991 movie finished
212095 movie finished
214553 movie finished
189137 movie finished
154282 movie finished
183862 movie finished
191020 movie finished
182016 movie finished
190374 movie finished
192608 movie finished
201701 movie finished
201938 movie finished
196854 movie finished
209859 movie finished
196362 movie finished
212098 movie finished
213733 movie finished
201766 movie finished
159812 movie finished
213903 movie finished
204138 movie finished
193961 movie finished
215637 movie finished
211165 movie finished
154255 movie finished
215791 movie finished
198624 movie finished
207444 movie finished
214569 movie finished
115642 movie finished
196051 movie finished
213580 movie finished
213570 movie finished
204650 movie finished
154485 movie finished
194196 movie finished
208077 movie finished
199164 movie finished
198413 movie finished
212159 movie finished
17059 movie finished
214665 movie finished
198501 movie finished
191613 movi

In [57]:
movie_final

[['164122',
  '신비한 동물들과 덤블도어의 비밀',
  ['판타지,', '모험,', '가족'],
  ['미국'],
  6.47,
  5.67,
  '142분 '],
 ['190695', '모비우스', ['액션,', '모험,', '드라마'], ['미국'], 7.22, 5.17, '104분 '],
 ['190991', '이상한 나라의 수학자', ['드라마'], ['한국'], 8.27, 5.43, '117분 '],
 ['212095',
  '수퍼 소닉2',
  ['애니메이션,', '액션,', '모험,', '코미디,', '가족,', '판타지,', 'SF'],
  ['미국'],
  8.27,
  7.0,
  '122분 '],
 ['214553', '앰뷸런스', ['액션,', '범죄'], ['미국'], 7.12, 6.0, '136분 '],
 ['189137', '스텔라', ['코미디,', '드라마'], ['한국'], 6.23, 5.0, '98분 '],
 ['154282', '더 배트맨', ['액션,', '범죄,', '드라마'], ['미국'], 7.24, 7.3, '176분 '],
 ['183862', '뜨거운 피', ['범죄,', '드라마'], ['한국'], 7.45, 5.5, '120분 '],
 ['191020', '야차', ['액션,', '모험'], ['한국'], 5.24, 0.0, 'NaN'],
 ['182016',
  '닥터 스트레인지: 대혼돈의 멀티버스',
  ['액션,', '판타지,', '모험'],
  ['미국'],
  9.92,
  0.0,
  '126분 '],
 ['190374', '앵커', ['스릴러'], ['한국'], 6.96, 5.25, '111분 '],
 ['192608', '범죄도시2', ['범죄,', '액션'], ['한국'], 0.0, 0.0, '106분 '],
 ['201701', '불도저에 탄 소녀', ['드라마'], ['한국'], 8.06, 5.5, '112분 '],
 ['201938', '그리스도 디 오리진', ['드라마'], 

In [58]:
import pandas as pd

columns = ['id_movie', 'title', 'genre', 'nation', 'audience_score', 'critic_score', 'runtime']
df = pd.DataFrame(movie_final, columns=columns)

In [59]:
df

,id_movie,title,genre,nation,audience_score,critic_score,runtime
0,164122,신비한 동물들과 덤블도어의 비밀,"[판타지,, 모험,, 가족]",[미국],6.47,5.67,142분
1,190695,모비우스,"[액션,, 모험,, 드라마]",[미국],7.22,5.17,104분
2,190991,이상한 나라의 수학자,[드라마],[한국],8.27,5.43,117분
3,212095,수퍼 소닉2,"[애니메이션,, 액션,, 모험,, 코미디,, 가족,, 판타지,, SF]",[미국],8.27,7.0,122분
4,214553,앰뷸런스,"[액션,, 범죄]",[미국],7.12,6.0,136분
5,189137,스텔라,"[코미디,, 드라마]",[한국],6.23,5.0,98분
6,154282,더 배트맨,"[액션,, 범죄,, 드라마]",[미국],7.24,7.3,176분
7,183862,뜨거운 피,"[범죄,, 드라마]",[한국],7.45,5.5,120분
8,191020,야차,"[액션,, 모험]",[한국],5.24,0.0,NaN
9,182016,닥터 스트레인지: 대혼돈의 멀티버스,"[액션,, 판타지,, 모험]",[미국],9.92,0.0,126분


In [60]:
df.to_csv('movie_final.csv', encoding='euc-kr')

# 2. 동적크롤링

In [45]:
!pip install selenium

In [79]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [80]:
driver = webdriver.Chrome()

In [81]:
driver.get('https://www.naver.com/')

In [82]:
hotel1 = driver.find_element(By.CLASS_NAME, '_3Apve')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3Apve"}
  (Session info: chrome=100.0.4896.88)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CD7413+2389011]
	Ordinal0 [0x00C69F61+1941345]
	Ordinal0 [0x00B5C658+837208]
	Ordinal0 [0x00B891DD+1020381]
	Ordinal0 [0x00B8949B+1021083]
	Ordinal0 [0x00BB6032+1204274]
	Ordinal0 [0x00BA4194+1130900]
	Ordinal0 [0x00BB4302+1196802]
	Ordinal0 [0x00BA3F66+1130342]
	Ordinal0 [0x00B7E546+976198]
	Ordinal0 [0x00B7F456+980054]
	GetHandleVerifier [0x00E89632+1727522]
	GetHandleVerifier [0x00F3BA4D+2457661]
	GetHandleVerifier [0x00D6EB81+569713]
	GetHandleVerifier [0x00D6DD76+566118]
	Ordinal0 [0x00C70B2B+1968939]
	Ordinal0 [0x00C75988+1989000]
	Ordinal0 [0x00C75A75+1989237]
	Ordinal0 [0x00C7ECB1+2026673]
	BaseThreadInitThunk [0x777FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77C07A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77C07A4E+238]


## 2. 동적크롤링: Selenium 기초
- selenium을 사용하려면 웹브라우저와 호환되는 드라이버 설치 필요
- chrome://version에서 크롬 버전 확인
- chromedriver에서 다운로드

- driver.back()
- driver.forward()
- driver.close()
- driver.quit()



- driver.find_element(By.CLASS_NAME, '000')
- driver.find_element(By.TAG_NAME, '000')
- driver.find_element(By.LINK_TEXT,'000')
- driver.find_element(By.XPATH,'000')
- driver.find_element(By.ID, 'loginForm')



- driver.find_elements(By.CLASS_NAME, '000')
- driver.find_elements(By.TAG_NAME, '000')
- driver.find_elements(By.LINK_TEXT,'000')
- driver.find_elements(By.XPATH,'000')
- driver.find_elements(By.ID, 'loginForm')



- elem.click()
- elem.send_keys()



- Keys.ENTER
- Keys.END


In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

#### webdriver 와 element를 이용하여 웹사이트 탐색
- 로그인 클릭
- 검색어 검색
- 드라이버 method

In [143]:
driver = webdriver.Chrome()

In [144]:
driver.get('http://naver.com')

In [26]:
elem = driver.find_element(By.CLASS_NAME, 'link_login') # class 값이 'link_login'인 element 가져오기  

In [27]:
elem.click()

In [28]:
elem = driver.find_element(By.ID, 'id')

In [29]:
elem.send_keys('naver_id')

In [31]:
elem = driver.find_element(By.ID, 'pw')

In [32]:
elem.send_keys('password')

In [23]:
driver.find_element(By.ID, 'log.login').click()

In [33]:
driver.back()

In [34]:
driver.forward()

In [35]:
driver.close()

In [42]:
driver.execute_script('window.scrollTo(0,1080)')

In [41]:
driver.execute_script('window.scroll.To(0,2080)')

JavascriptException: Message: javascript error: window.scroll.To is not a function
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x004F7413+2389011]
	Ordinal0 [0x00489F61+1941345]
	Ordinal0 [0x0037C658+837208]
	Ordinal0 [0x0037F064+847972]
	Ordinal0 [0x0037EF22+847650]
	Ordinal0 [0x0037F89A+850074]
	Ordinal0 [0x003D4D49+1199433]
	Ordinal0 [0x003C414C+1130828]
	Ordinal0 [0x003D4302+1196802]
	Ordinal0 [0x003C3F66+1130342]
	Ordinal0 [0x0039E546+976198]
	Ordinal0 [0x0039F456+980054]
	GetHandleVerifier [0x006A9632+1727522]
	GetHandleVerifier [0x0075BA4D+2457661]
	GetHandleVerifier [0x0058EB81+569713]
	GetHandleVerifier [0x0058DD76+566118]
	Ordinal0 [0x00490B2B+1968939]
	Ordinal0 [0x00495988+1989000]
	Ordinal0 [0x00495A75+1989237]
	Ordinal0 [0x0049ECB1+2026673]
	BaseThreadInitThunk [0x777FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77C07A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77C07A4E+238]


### XPath
- XML 문서의 특정 요소나 속성에 접근하기 위한 경로를 지정하는 언어
- 특정한 값을 이용해 경로를 간단하게 표현

In [145]:
path = '//*[@id="NM_FAVORITE"]/div[1]/ul[2]/li[2]/a'

In [146]:
elem = driver.find_element(By.XPATH, path)

In [147]:
elem.click()

In [ ]:
1

### EXAMPLE 1) 네이버지도 
- 네이버지도에서 서울의 호텔에 대한 데이터 가져오기

In [112]:
import time

In [148]:
URL = 'https://map.naver.com/'
KEY_WORD = '서울 호텔'
SEC=3

In [149]:
driver = webdriver.Chrome()
driver.get(URL)
time.sleep(SEC)
input_box = driver.find_element(By.CLASS_NAME, value='input_search')
time.sleep(SEC)
input_box.send_keys(KEY_WORD)
input_box.send_keys(Keys.ENTER)
time.sleep(SEC)

#### 1) iframe 태그 주의
- iframe은 html 웹 문서 안에 또 다른 웹 문서 혹은 자료를 넣을 때 사용
- 정적 크롤링 방식으로는 iframe 결과를 가져올 수 없음
- selenium을 이용해서 frame 변환 가능

In [150]:
list_hotel = driver.find_elements(By.CLASS_NAME, '_3Apve')

In [158]:
len(list_hotel)

20

In [159]:
elem = driver.find_element(By.ID, 'searchIframe')
driver.switch_to.frame(elem)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="searchIframe"]"}
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x004F7413+2389011]
	Ordinal0 [0x00489F61+1941345]
	Ordinal0 [0x0037C658+837208]
	Ordinal0 [0x003A91DD+1020381]
	Ordinal0 [0x003A949B+1021083]
	Ordinal0 [0x003D6032+1204274]
	Ordinal0 [0x003C4194+1130900]
	Ordinal0 [0x003D4302+1196802]
	Ordinal0 [0x003C3F66+1130342]
	Ordinal0 [0x0039E546+976198]
	Ordinal0 [0x0039F456+980054]
	GetHandleVerifier [0x006A9632+1727522]
	GetHandleVerifier [0x0075BA4D+2457661]
	GetHandleVerifier [0x0058EB81+569713]
	GetHandleVerifier [0x0058DD76+566118]
	Ordinal0 [0x00490B2B+1968939]
	Ordinal0 [0x00495988+1989000]
	Ordinal0 [0x00495A75+1989237]
	Ordinal0 [0x0049ECB1+2026673]
	BaseThreadInitThunk [0x777FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77C07A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77C07A4E+238]


In [160]:
list_hotel = driver.find_elements(By.CLASS_NAME, '_3Apve')

In [161]:
list_hotel

[<selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="e6eeb8fc-0c7a-4e5c-86b3-c1d7fc171fc9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="601f572a-8d1c-4b9a-ba0e-b4cabe75d45f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="90414705-4b22-4f2c-8546-a9a193afdc5a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="492fe88b-d73a-4f11-bbfd-b1d761791eb9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="e4aa1dab-2c59-494e-9e41-dc2f8c3ad1fe")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="fa86f61e-0e1e-4d07-adbc-a9c323e3377f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="0b6201b8-f8c8-4854-a1b5-d9

In [162]:
len(list_hotel)

50

#### 2) lazy loading
- 한번에 모든 데이터를 보여주지 않는 경우가 있을 수 있음
- 네이버지도의 경우 스크롤을 끝까지 내려야지 모든 정보를 받아올 수 있음

In [163]:
# 스크롤 내려야 하는 곳 = 맨밑
elem = driver.find_element(By.CLASS_NAME, 'place_on_pcmap')

In [164]:
elem.click()

In [165]:
elem.send_keys(Keys.END)

In [166]:
list_hotel = driver.find_elements(By.CLASS_NAME, '_3Apve')
len(list_hotel)

50

#### 3) 정보 가져오기

In [167]:
hotel1 = driver.find_element(By.CLASS_NAME, '_3Apve')

In [168]:
hotel1.click()

In [169]:
# 호텔 정보를 가져오기 위해 parent frame으로 나갔다가 entrylframe 으로 들어감

driver.switch_to.parent_frame()
driver.switch_to.frame('entryIframe')

In [170]:
# 더보기 버튼을 클릭하기 위해 element 들을 가져옴

In [171]:
list_button = driver.find_elements(By.CLASS_NAME, '_3iTUo')

In [172]:
list_button

[<selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="84828c0a-95ca-4708-b61e-ec8c5b9e9393")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="268dc5c3-392d-4e4b-b1a8-f568f952b3d7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="f1137a83-a789-4c10-802d-f6a3249315b7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="1e9f3c7a-9ae7-4dae-8c5d-677d7545c38f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0123f715ad8b67ce442cc5a7b873d2ec", element="20a5a2bf-8840-4a35-b711-e591dd2d9378")>]

In [173]:
for button in list_button:
    print(button.text)

실시간 가격비교더보기
방문자 사진더보기
방문자 리뷰더보기
더보기
블로그리뷰더보기


In [175]:
list_button[3].click()

In [183]:
html = driver.page_source
soup= BeautifulSoup(html, 'lxml')

In [184]:
1

1

- 호텔 이름

In [185]:
name = soup.find('span', {'class':'_3XamX'}).text
name

'그랜드 하얏트 서울'

- 몇성급인지

In [186]:
star = soup.select_one('#_title > span._3ocDE').text
star

'5성급'

- 별점

In [187]:
try:
    score = float(soup.find('span', {'class':'_1Y6hi _1A8_M'}).em.text)
except:
    score = 'NaN'
score

4.49

- 전화번호

In [188]:
phone = soup.select_one('#app-root > div > div > div > div:nth-child(5) > div > div.place_section.no_margin._18vYz > div > ul > li._1M_Iz._3xPmJ > div > span._3ZA0S').text
phone

AttributeError: 'NoneType' object has no attribute 'text'

- 주소

In [189]:
address = soup.select_one('#app-root > div > div > div > div:nth-child(5) > div > div.place_section.no_margin._18vYz > div > ul > li._1M_Iz._1aj6- > div > a > span._2yqUQ').text
address

AttributeError: 'NoneType' object has no attribute 'text'

- 평점

In [ ]:
def extract_text(list_tag):
    result = [tag.text.strip() for tag in list_tag]
    return result

In [ ]:
rating = extract_text(soup.select('#app-root > div > div > div > div:nth-child(5) > div > div.place_section.no_margin._18vYz > div > ul > li.jQ772.D_Ode > div > div > span'))
rating

- 가격정보

In [ ]:
list_price = soup.find_all(class_='_341sT')

In [ ]:
list_price[0].select_one('a > div > em').text

In [ ]:
if len(list_price) >= 1:
    price_total = list_price[0].select_one('a > div > em').text
if len(list_price) >= 2:
    price_package = list_price[1].select_one('a > div > em').text

In [ ]:
price_total

In [ ]:
price_package

- 시설

In [ ]:
temp = soup.select('#app-root > div > div > div.place_detail_wrapper > div > div > div > div.place_section_content > div > div > ul > li')
facilities = extract_text(temp)
facilities

- 정보 종합

In [ ]:
info = [name, star, score, phone, address, rating, price_total, price_package, facilities]

In [ ]:
info